In [132]:
import time

EMPTY = 0
MAJOON = 1
TWO = 2
ROCK = 3

dir = [[0, 1], [0,-1], [-1, 0], [1, 0]]
UP = [0, 1]
DOWN = [0, -1]
RIGHT = [1, 0]
LEFT = [-1, 0]
END = "end"

In [198]:
inputFile = open("test1.in")

n, m = (int(inputf) for inputf in inputFile.readline().split())
c, k = (int(inputf) for inputf in inputFile.readline().split())

table = [[EMPTY for i in range(n)] for j in range(m)]
majoon = []

for i in range(c):
    y, x = (int(inputf) for inputf in inputFile.readline().split())
    table[x][y] = MAJOON
    majoon.append([x, y])

for i in range(k):
    y, x = (int(inputf) for inputf in inputFile.readline().split())
    table[x][y] = TWO

d = int(inputFile.readline())
for i in range(d):
    y, x = (int(inputf) for inputf in inputFile.readline().split())
    table[x][y] = ROCK


inputFile.close()

In [134]:
def checkCell(x, y):
    if(x < 0 or x >= m or y < 0 or y >= n):
        return False
    elif(table[x][y] == ROCK):
        return False
    else:
        return True

def checkGoal(currState, num):
    goalX = str(n-1)
    goalY = str(m-1)
    for i in range(num):
        if(currState[4*i] != goalX or currState[4*i + 2] != goalY):
            return False
    if(countPotion(currState) == c):
        return True
    else:
        return False

def makeState(str1, n, s0, s1, d):
    idx = str1.index("|")
    lstr = [str(e) for e in str1[:idx].split()]
    lstr[(n*2)] = str(s0 + d[0])
    lstr[n*2+1] = str(s1 + d[1])
    str2 = ""
    for i in range(len(lstr)):
        str2 += lstr[i] + " "
    str2 += str1[idx:]  
    if(table[s0+d[0]][s1+d[1]] == MAJOON):                 
        str2 = addPotion(str2, s0+d[0], s1+d[1])                    
    elif table[s0+d[0]][s1+d[1]] == TWO:
        str2 = addDoctor(str2, s0+d[0], s1+d[1]) 
    return str2

def makePosition(curr, n):
    position = curr.split()
    s0 = int(position[(n)*2])
    s1 = int(position[(n)*2 + 1])
    return s0, s1

def addPotion(curr, x, y):
    idx = curr.index("|")
    idx2 = curr.index("|", idx + 1)
    if(curr.find(str(x) + " " + str(y),idx, idx2) != -1):
        return curr
    curr = curr[:idx2] + str(x) + " " + str(y) + " " + curr[idx2:]  
    return curr

def addDoctor(curr, x, y):
    idx = curr.index("|")
    idx1 = curr.rindex("|")
    if curr.find(str(x)+" "+str(y), idx1) == -1:
        curr += str(x)+" "+ str(y)+" "
        curr = curr[:idx] + "0 " + str(n-1) + " " + curr[idx:]
    return curr

def countDoctor(curr):
    idx = curr.index("|")
    str1 = curr[:idx]
    return int((len(str1))/4)

def countPotion(curr):
    idx = curr.index("|")
    idx2 = curr.rindex("|")
    return int((idx2 - idx)/4)

In [149]:
def BFS(table):
    curr = "0 0 | | " #position doctor[1] position[2] ...| potion 1 potion 2 ... | drug 1 drug 2
    if(table[0][0] == MAJOON):                 
        curr = addPotion(curr, 0, 0)
    elif table[0][0] == TWO:
        curr = addDoctor(curr, 0, 0)
    step = 0
    states = set()
    queue = set()
    queue.add(curr)
    fatherDic = {curr : END}
    while queue: 
        curr = queue.pop()
        states.add(curr)  
        numberOfDoctors = countDoctor(curr)
        
        for numDoctor in range(numberOfDoctors):
            s0, s1 = makePosition(curr, numDoctor)
            for d in dir:
                step += 1        
                if not checkCell(s0+d[0], s1+d[1]):
                    continue
                currState = makeState(curr, numDoctor, s0, s1, d)
                
                if(checkGoal(currState, numberOfDoctors)):
                    step += 1
                    fatherDic[currState] = curr
                    return fatherDic, currState, step                                 

                if (currState not in states and currState not in queue):  
                    queue.add(currState)  
                    fatherDic[currState] = curr

In [155]:
def findPos(cell, n):
    if(countDoctor(cell)-1 < n):
        return -100, -100, END
    return int(cell[4*n]), int(cell[4*n+2]),cell

def printPath(path, finalState):
    numDoctors = countDoctor(finalState)
    step = 0
    for i in range(numDoctors):
        pathArray = []
        curr = finalState
        currFather = path[curr]
        cellx1, celly1, curr = findPos(curr, i)
        while(curr != END):
            if(currFather == END):
                cellx2, celly2 = 0, 0
            else:
                cellx2, celly2,currFather = findPos(currFather, i)
            way = [cellx1 - cellx2, celly1 - celly2]
            if(way == UP):
                pathArray.append("U"+str(i))
            elif(way == DOWN):
                pathArray.append("D"+str(i))
            elif(way == RIGHT):
                pathArray.append("R"+str(i))
            elif(way == LEFT):
                pathArray.append("L"+str(i))
                
            if(currFather == END):
                pathArray.reverse()
                print(pathArray)
                step += len(pathArray)
                break
            curr = currFather
            currFather = path[curr]
            cellx1, celly1 = cellx2, celly2
    print("step to the answer", step)

In [163]:
def IDDFS(table):
    curr = "0 0 | | " #position doctor[1] position[2] ...| potion 1 potion 2 ... | drug 1 drug 2
    if(table[0][0] == MAJOON):                 
        curr = addPotion(curr, 0, 0)
    elif table[0][0] == TWO:
        curr = addDoctor(curr, 0, 0)
    fatherDic = {curr : END}
    for depth in range(n*m):
        queue = set()
        queue.add(curr)
        #print("depth input", depth)
        found = IDS(curr,curr, depth, queue, fatherDic)
        if found:
            return fatherDic, found

def IDS(curr, currFather, depth, queue, fatherDic):
    #print(curr)
    #print("depth", depth)
    numberOfDoctors = countDoctor(curr)
    
    if depth < 0:
        if(checkGoal(curr, numberOfDoctors)):
            fatherDic[currFather] = curr
            return curr
        else: 
            return None 
    for numDoctor in range(numberOfDoctors):
        s0, s1 = makePosition(curr, numDoctor)
        for d in dir:
            if not checkCell(s0+d[0], s1+d[1]):
                continue
            currState = makeState(curr, numDoctor, s0, s1, d)      

            if (currState not in queue):
                found = IDS(currState,curr, depth-1, queue, fatherDic)
                if found:
                    fatherDic[currState] = curr
                    return found
                queue.add(found)  

In [167]:
def findMinF(openList, f):
    minimum = f[openList[0]]
    curr = openList[0]
    for i in range(len(openList)):
        if(f[openList[i]] < minimum):
            minimum = f[openList[i]]
            curr = openList[i]
    return curr

def calF(curr):
    h = 0
    g = 0
    numDoctor = countDoctor(curr)
    for i in range(numDoctor):
        x0, y0 = makePosition(curr, i)
        h += n - x0 + m - y0
        x1, y1 = makePosition(curr, i)
        g += x1 + y1
    return h + g

def AStar(table):
    first = "0 0 | | " #position doctor[1] position[2] ...| potion 1 potion 2 ... | drug 1 drug 2
    if(table[0][0] == MAJOON):                 
        first = addPotion(first, 0, 0)
    elif table[0][0] == TWO:
        first = addDoctor(first, 0, 0)
    fatherDic = {first : END}

    openList = []
    closedList = set()

    step = 0
    f = {}
    openList.append(first)
    f[first] = 0
    while(openList):
        curr = findMinF(openList, f)
        openList.remove(curr)
        closedList.add(curr)
        numberOfDoctors = countDoctor(curr)
        if(checkGoal(curr, numberOfDoctors)):
            return fatherDic, curr, step

        numberOfDoctors = countDoctor(curr)
        for numDoctor in range(numberOfDoctors):
            s0, s1 = makePosition(curr, numDoctor)
            for d in dir:
                step += 1
                if not checkCell(s0+d[0], s1+d[1]):
                    continue
                
                currState = makeState(curr, numDoctor, s0, s1, d)      
                if (currState in closedList):
                    continue
                f[currState] = calF(currState)
                fatherDic[currState] = curr
                openList.append(currState)
    return None
    

In [177]:

def AweightedStar(table):
    first = "0 0 | | " #position doctor[1] position[2] ...| potion 1 potion 2 ... | drug 1 drug 2
    if(table[0][0] == MAJOON):                 
        first = addPotion(first, 0, 0)
    elif table[0][0] == TWO:
        first = addDoctor(first, 0, 0)
    fatherDic = {first : END}

    openList = []
    closedList = set()
    step = 0
    f = {}
    openList.append(first)
    f[first] = 0
    while(openList):
        curr = findMinF(openList, f)
        openList.remove(curr)
        closedList.add(curr)
        numberOfDoctors = countDoctor(curr)
        if(checkGoal(curr, numberOfDoctors)):
            return fatherDic, curr, step

        numberOfDoctors = countDoctor(curr)
        for numDoctor in range(numberOfDoctors):
            s0, s1 = makePosition(curr, numDoctor)
            for d in dir:
                step += 1
                if not checkCell(s0+d[0], s1+d[1]):
                    continue
                
                currState = makeState(curr, numDoctor, s0, s1, d)      
                if (currState in closedList):
                    continue
                #a = 1.5
                a = 1.5
                f[currState] = a*calF(currState)
                fatherDic[currState] = curr
                openList.append(currState)
    return None

In [199]:
start = time.time()
#path, finalState, numberOfStates = BFS(table)
#path, finalState = IDDFS(table)
#path, finalState, numberOfStates = AStar(table)
path, finalState, numberOfStates = AweightedStar(table)
print("Number of State",numberOfStates)
end = time.time()
print(f"Elapsed Time: {end - start} Seconds")
printPath(path, finalState)


Number of State 172
Elapsed Time: 0.002775907516479492 Seconds
['U0', 'U0', 'R0', 'R0', 'R0', 'D0', 'D0', 'L0', 'R0', 'U0', 'U0', 'U0']
step to the answer 12
